In [1]:
# CELL 1: Setup
import pandas as pd
from pathlib import Path
import nibabel as nib
import numpy as np
from scipy.ndimage import label, center_of_mass

CSV_FILE = Path('/user_data/csimmon2/git_repos/long_pt/long_pt_sub_info.csv')
df = pd.read_csv(CSV_FILE)

BASE_DIR = Path("/user_data/csimmon2/long_pt")
SESSION_START = {'sub-010': 2, 'sub-018': 2, 'sub-068': 2}

'''
COPE_MAP = {
    'face':   (10, 1),
    'word':   (13, -1),
    'object': (3,  1),
    'house':  (11, 1)
}
'''

# temp:

COPE_MAP = {
    'face': (10, 1),   # Face > Scramble
    'word': (12, 1),   # Word > Scramble
    'object': (3, 1),  # Object > Scramble
    'house': (11, 1)   # House > Scramble
}

In [2]:
# CELL 2: Load Subjects
def load_subjects_by_group(group_filter=None, patient_only=True):
    filtered_df = df.copy()
    
    if patient_only is True:
        filtered_df = filtered_df[filtered_df['patient'] == 1]
    elif patient_only is False:
        filtered_df = filtered_df[filtered_df['patient'] == 0]
    
    if group_filter:
        if isinstance(group_filter, str):
            group_filter = [group_filter]
        filtered_df = filtered_df[filtered_df['group'].isin(group_filter)]
    
    subjects = {}
    for _, row in filtered_df.iterrows():
        subject_id = row['sub']
        subj_dir = BASE_DIR / subject_id
        if not subj_dir.exists():
            continue
        
        sessions = sorted([d.name.replace('ses-', '') for d in subj_dir.glob('ses-*') if d.is_dir()], key=int)
        start_session = SESSION_START.get(subject_id, 1)
        sessions = [s for s in sessions if int(s) >= start_session]
        if not sessions:
            continue
        
        hemisphere = 'l' if row.get('intact_hemi', 'left') == 'left' else 'r'
        
        subjects[subject_id] = {
            'code': f"{row['group']}{subject_id.split('-')[1]}",
            'sessions': sessions,
            'hemi': hemisphere,
            'group': row['group'],
            'patient_status': 'patient' if row['patient'] == 1 else 'control',
            'surgery_side': row.get('SurgerySide', None)
        }
    return subjects

ALL_PATIENTS = load_subjects_by_group(patient_only=True)
ALL_CONTROLS = load_subjects_by_group(patient_only=False)
ANALYSIS_SUBJECTS = {**ALL_PATIENTS, **ALL_CONTROLS}

In [3]:
# CELL 3: ROI Extraction Function
def extract_functional_rois_bilateral(subject_id, threshold_z=2.3, min_cluster_size=30):
    info = ANALYSIS_SUBJECTS[subject_id]
    roi_dir = BASE_DIR / subject_id / f'ses-{info["sessions"][0]}' / 'ROIs'
    if not roi_dir.exists(): 
        return {}
    
    all_results = {}
    first_session = info['sessions'][0]

    for hemi in ['l', 'r']:
        for category, cope_params in COPE_MAP.items():
            cope_num, multiplier = cope_params
            
            mask_file = roi_dir / f'{hemi}_{category}_searchmask.nii.gz'
            if not mask_file.exists(): 
                continue
            
            try:
                search_mask = nib.load(mask_file).get_fdata() > 0
                affine = nib.load(mask_file).affine
            except: 
                continue
            
            hemi_key = f'{hemi}_{category}'
            all_results[hemi_key] = {}
            
            for session in info['sessions']:
                feat_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
                z_name = 'zstat1.nii.gz' if session == first_session else f'zstat1_ses{first_session}.nii.gz'
                cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / z_name
                
                if not cope_file.exists(): 
                    continue
                
                try:
                    zstat = nib.load(cope_file).get_fdata() * multiplier
                    suprathresh = (zstat > threshold_z) & search_mask
                    labeled, n_clusters = label(suprathresh)
                    if n_clusters == 0: 
                        continue
                    
                    best_idx, max_peak = -1, -999
                    for i in range(1, n_clusters + 1):
                        cluster_mask = (labeled == i)
                        if np.sum(cluster_mask) >= min_cluster_size:
                            peak_val = np.max(zstat[cluster_mask])
                            if peak_val > max_peak:
                                max_peak, best_idx = peak_val, i
                    
                    if best_idx == -1: 
                        continue
                    
                    roi_mask = (labeled == best_idx)
                    peak_idx = np.unravel_index(np.argmax(zstat * roi_mask), zstat.shape)
                    
                    all_results[hemi_key][session] = {
                        'n_voxels': int(np.sum(roi_mask)),
                        'peak_z': zstat[peak_idx],
                        'centroid': nib.affines.apply_affine(affine, center_of_mass(roi_mask)),
                        'roi_mask': roi_mask
                    }
                except Exception as e: 
                    print(f"Err {subject_id} {category}: {e}")
    return all_results

In [4]:
# CELL 4: Extract All Subjects
print("Extracting ROIs...")
golarai_functional_final = {}
for sub in ANALYSIS_SUBJECTS:
    res = extract_functional_rois_bilateral(sub, min_cluster_size=30)
    if res: 
        golarai_functional_final[sub] = res
print(f"✓ Done: {len(golarai_functional_final)} subjects")

Extracting ROIs...
✓ Done: 23 subjects


In [5]:
# CELL 5: OTC017 Diagnostic
sid = 'sub-017'
info = ANALYSIS_SUBJECTS[sid]
print(f"Subject: {info['code']}")
print(f"Resection side: {info.get('surgery_side', 'unknown')}")
print(f"Intact hemi: {info['hemi']}")
print(f"Sessions: {info['sessions']}")

print("\nAll ROIs:")
if sid in golarai_functional_final:
    for roi_key, sessions in golarai_functional_final[sid].items():
        print(f"\n{roi_key}:")
        for ses, data in sorted(sessions.items()):
            print(f"  ses-{ses}: n={data['n_voxels']}, z={data['peak_z']:.2f}, centroid={[round(c,1) for c in data['centroid']]}")

Subject: OTC017
Resection side: left
Intact hemi: r
Sessions: ['01', '02', '03', '04']

All ROIs:

r_face:
  ses-01: n=1568, z=9.13, centroid=[39.1, -41.8, -5.5]
  ses-02: n=1505, z=9.42, centroid=[39.4, -46.5, -4.1]
  ses-03: n=1231, z=8.23, centroid=[39.4, -46.3, -3.5]
  ses-04: n=2143, z=10.63, centroid=[37.2, -29.3, -11.2]

r_word:
  ses-01: n=1059, z=5.81, centroid=[37.9, -57.8, 0.1]
  ses-02: n=886, z=5.66, centroid=[39.5, -51.3, -3.6]
  ses-03: n=75, z=4.54, centroid=[35.3, -77.3, -8.0]
  ses-04: n=1681, z=6.14, centroid=[36.4, -29.0, -10.0]

r_object:
  ses-01: n=3362, z=9.04, centroid=[46.1, -88.5, 11.2]
  ses-02: n=6756, z=8.67, centroid=[42.7, -86.2, 2.5]
  ses-03: n=10641, z=10.93, centroid=[41.2, -90.0, 3.7]
  ses-04: n=7908, z=10.32, centroid=[44.3, -88.3, 3.6]

r_house:
  ses-01: n=2147, z=10.32, centroid=[28.6, -35.3, 3.4]
  ses-02: n=962, z=5.39, centroid=[31.1, -34.3, 3.2]
  ses-03: n=1225, z=5.60, centroid=[29.7, -35.5, 3.5]
  ses-04: n=2454, z=11.39, centroid=[26.3,

In [6]:
# CELL 6: Word-specific drift check
print("\nWord Drift Details:")
for key in golarai_functional_final.get(sid, {}):
    if 'word' in key:
        sessions = sorted(golarai_functional_final[sid][key].keys())
        if len(sessions) >= 2:
            c1 = golarai_functional_final[sid][key][sessions[0]]['centroid']
            c2 = golarai_functional_final[sid][key][sessions[-1]]['centroid']
            print(f"T1 centroid: {[round(c,1) for c in c1]}")
            print(f"T2 centroid: {[round(c,1) for c in c2]}")
            print(f"Drift: {np.linalg.norm(np.array(c2)-np.array(c1)):.1f}mm")


Word Drift Details:
T1 centroid: [37.9, -57.8, 0.1]
T2 centroid: [36.4, -29.0, -10.0]
Drift: 30.5mm


In [7]:
# CELL 7: Check all subjects for instability
def check_cluster_stability(functional_results, subjects_dict):
    """Flag subjects with potential cluster selection issues"""
    
    flags = []
    
    for sid, rois in functional_results.items():
        info = subjects_dict.get(sid, {})
        code = info.get('code', sid)
        
        for roi_key, sessions_data in rois.items():
            sessions = sorted(sessions_data.keys())
            if len(sessions) < 2:
                continue
            
            # Get metrics across sessions
            voxels = [sessions_data[s]['n_voxels'] for s in sessions]
            peaks = [sessions_data[s]['peak_z'] for s in sessions]
            centroids = [sessions_data[s]['centroid'] for s in sessions]
            
            # Flag 1: Tiny T1 cluster (<150 voxels)
            if voxels[0] < 150:
                flags.append({
                    'subject': code, 'roi': roi_key, 'issue': 'tiny_T1',
                    'detail': f'n={voxels[0]}'
                })
            
            # Flag 2: Weak T1 activation (z < 4.0)
            if peaks[0] < 4.0:
                flags.append({
                    'subject': code, 'roi': roi_key, 'issue': 'weak_T1',
                    'detail': f'z={peaks[0]:.2f}'
                })
            
            # Flag 3: Large voxel count swing (>5x change)
            if max(voxels) / min(voxels) > 5:
                flags.append({
                    'subject': code, 'roi': roi_key, 'issue': 'voxel_swing',
                    'detail': f'{min(voxels)}→{max(voxels)}'
                })
            
            # Flag 4: Y-coordinate oscillation (>15mm back-and-forth)
            if len(sessions) >= 3:
                y_coords = [c[1] for c in centroids]
                for i in range(len(y_coords)-2):
                    if abs(y_coords[i] - y_coords[i+1]) > 15 and abs(y_coords[i+1] - y_coords[i+2]) > 15:
                        if (y_coords[i] - y_coords[i+1]) * (y_coords[i+1] - y_coords[i+2]) < 0:  # opposite directions
                            flags.append({
                                'subject': code, 'roi': roi_key, 'issue': 'oscillation',
                                'detail': f'Y: {[round(y,1) for y in y_coords]}'
                            })
                            break
    
    return pd.DataFrame(flags)

flags_df = check_cluster_stability(golarai_functional_final, ANALYSIS_SUBJECTS)
print("STABILITY FLAGS:")
print(flags_df.to_string(index=False))

STABILITY FLAGS:
   subject      roi       issue                          detail
    OTC004   l_face     weak_T1                          z=3.50
    OTC004   l_face voxel_swing                        352→4282
    OTC004   l_word oscillation Y: [-61.1, -42.8, -62.8, -37.0]
    OTC008 l_object voxel_swing                        293→8723
    OTC010   r_word voxel_swing                          33→864
    OTC010  r_house     tiny_T1                            n=33
    OTC010  r_house     weak_T1                          z=3.92
    OTC017   r_word voxel_swing                         75→1681
    OTC017   r_word oscillation Y: [-57.8, -51.3, -77.3, -29.0]
 nonOTC081   r_word voxel_swing                          31→212
control022   l_face voxel_swing                        695→3628
control022   r_word voxel_swing                        235→1588
control027   l_word voxel_swing                        439→3137
control027 l_object voxel_swing                      1785→13036
control027   r_word vox

In [8]:
# CELL 8: Cluster tracking with spatial overlap
def extract_with_tracking(subject_id, threshold_z=2.3, min_cluster_size=30, min_overlap=0.3):
    """Require spatial overlap with T1 ROI for subsequent sessions"""
    
    info = ANALYSIS_SUBJECTS[subject_id]
    roi_dir = BASE_DIR / subject_id / f'ses-{info["sessions"][0]}' / 'ROIs'
    if not roi_dir.exists():
        return {}
    
    all_results = {}
    first_session = info['sessions'][0]

    for hemi in ['l', 'r']:
        for category, cope_params in COPE_MAP.items():
            cope_num, multiplier = cope_params
            
            mask_file = roi_dir / f'{hemi}_{category}_searchmask.nii.gz'
            if not mask_file.exists():
                continue
            
            try:
                search_mask = nib.load(mask_file).get_fdata() > 0
                affine = nib.load(mask_file).affine
            except:
                continue
            
            hemi_key = f'{hemi}_{category}'
            all_results[hemi_key] = {}
            t1_mask = None
            
            for session in info['sessions']:
                feat_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
                z_name = 'zstat1.nii.gz' if session == first_session else f'zstat1_ses{first_session}.nii.gz'
                cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / z_name
                
                if not cope_file.exists():
                    continue
                
                try:
                    zstat = nib.load(cope_file).get_fdata() * multiplier
                    suprathresh = (zstat > threshold_z) & search_mask
                    labeled, n_clusters = label(suprathresh)
                    if n_clusters == 0:
                        continue
                    
                    # For T1: pick largest cluster meeting threshold
                    if session == first_session:
                        best_idx, max_peak = -1, -999
                        for i in range(1, n_clusters + 1):
                            cluster_mask = (labeled == i)
                            if np.sum(cluster_mask) >= min_cluster_size:
                                peak_val = np.max(zstat[cluster_mask])
                                if peak_val > max_peak:
                                    max_peak, best_idx = peak_val, i
                        
                        if best_idx == -1:
                            continue
                        
                        roi_mask = (labeled == best_idx)
                        t1_mask = roi_mask.copy()
                    
                    # For subsequent: require overlap with T1
                    else:
                        if t1_mask is None:
                            continue
                        
                        best_idx, best_overlap = -1, 0
                        for i in range(1, n_clusters + 1):
                            cluster_mask = (labeled == i)
                            if np.sum(cluster_mask) < min_cluster_size:
                                continue
                            
                            # Calculate overlap with T1
                            overlap = np.sum(cluster_mask & t1_mask) / np.sum(t1_mask)
                            if overlap > best_overlap:
                                best_overlap, best_idx = overlap, i
                        
                        if best_idx == -1 or best_overlap < min_overlap:
                            print(f"  {subject_id} {hemi_key} ses-{session}: No overlapping cluster (best={best_overlap:.2f})")
                            continue
                        
                        roi_mask = (labeled == best_idx)
                    
                    peak_idx = np.unravel_index(np.argmax(zstat * roi_mask), zstat.shape)
                    
                    all_results[hemi_key][session] = {
                        'n_voxels': int(np.sum(roi_mask)),
                        'peak_z': zstat[peak_idx],
                        'centroid': nib.affines.apply_affine(affine, center_of_mass(roi_mask)),
                        'roi_mask': roi_mask
                    }
                except Exception as e:
                    print(f"Err {subject_id} {category}: {e}")
    
    return all_results

# Test on OTC017
print("OTC017 with cluster tracking:")
tracked_017 = extract_with_tracking('sub-017', min_overlap=0.3)

print("\nWord ROI comparison:")
print("Original:", end=" ")
orig = golarai_functional_final['sub-017'].get('r_word', {})
for s in sorted(orig.keys()):
    print(f"ses-{s}: {[round(c,1) for c in orig[s]['centroid']]}", end=" | ")

print("\nTracked: ", end=" ")
tracked = tracked_017.get('r_word', {})
for s in sorted(tracked.keys()):
    print(f"ses-{s}: {[round(c,1) for c in tracked[s]['centroid']]}", end=" | ")

OTC017 with cluster tracking:
  sub-017 r_word ses-03: No overlapping cluster (best=0.18)
  sub-017 r_word ses-04: No overlapping cluster (best=0.24)

Word ROI comparison:
Original: ses-01: [37.9, -57.8, 0.1] | ses-02: [39.5, -51.3, -3.6] | ses-03: [35.3, -77.3, -8.0] | ses-04: [36.4, -29.0, -10.0] | 
Tracked:  ses-01: [37.9, -57.8, 0.1] | ses-02: [39.5, -51.3, -3.6] | 

In [9]:
# CELL 9: Re-extract ALL subjects with tracking, then recalculate all metrics
print("Extracting with cluster tracking (all subjects)...")
tracked_functional = {}
for sid in ANALYSIS_SUBJECTS:
    res = extract_with_tracking(sid, min_overlap=0.3)
    if res:
        tracked_functional[sid] = res

print(f"✓ Done: {len(tracked_functional)} subjects")

# Compare: how many session-ROIs dropped?
original_count = sum(len(s) for rois in golarai_functional_final.values() for s in rois.values())
tracked_count = sum(len(s) for rois in tracked_functional.values() for s in rois.values())
print(f"Original session-ROIs: {original_count}")
print(f"Tracked session-ROIs: {tracked_count}")
print(f"Dropped: {original_count - tracked_count}")

Extracting with cluster tracking (all subjects)...
  sub-008 l_object ses-02: No overlapping cluster (best=0.07)
  sub-008 l_house ses-02: No overlapping cluster (best=0.20)
  sub-010 r_word ses-03: No overlapping cluster (best=0.00)
  sub-010 r_house ses-03: No overlapping cluster (best=0.00)
  sub-017 r_word ses-03: No overlapping cluster (best=0.18)
  sub-017 r_word ses-04: No overlapping cluster (best=0.24)
  sub-081 r_word ses-02: No overlapping cluster (best=0.15)
  sub-018 r_word ses-03: No overlapping cluster (best=0.27)
  sub-022 r_word ses-02: No overlapping cluster (best=0.00)
  sub-027 l_face ses-02: No overlapping cluster (best=0.24)
  sub-027 l_word ses-02: No overlapping cluster (best=0.13)
  sub-027 r_word ses-02: No overlapping cluster (best=0.10)
  sub-052 r_word ses-02: No overlapping cluster (best=0.24)
  sub-052 r_house ses-02: No overlapping cluster (best=0.24)
  sub-058 r_word ses-02: No overlapping cluster (best=0.00)
  sub-062 r_word ses-02: No overlapping clus

In [10]:
# CELL 10: RDM with Dynamic Spherical ROIs

def create_sphere(peak_coord, affine, brain_shape, radius=6):
    """Create sphere around peak coordinate"""
    grid_coords = np.array(np.meshgrid(
        np.arange(brain_shape[0]),
        np.arange(brain_shape[1]),
        np.arange(brain_shape[2]),
        indexing='ij'
    )).reshape(3, -1).T
    
    grid_world = nib.affines.apply_affine(affine, grid_coords)
    distances = np.linalg.norm(grid_world - peak_coord, axis=1)
    
    mask_3d = np.zeros(brain_shape, dtype=bool)
    within = grid_coords[distances <= radius]
    for coord in within:
        mask_3d[coord[0], coord[1], coord[2]] = True
    
    return mask_3d

# Two COPE maps to test
COPE_SCRAMBLE = {
    'face': (10, 1),   # Face > Scramble
    'word': (12, 1),   # Word > Scramble
    'object': (3, 1),  # Object > Scramble
    'house': (11, 1)   # House > Scramble
}

COPE_DIFFERENTIAL = {
    'face': (10, 1),   # Face > Scramble
    'word': (13, -1),  # Face > Word inverted
    'object': (3, 1),  # Object > Scramble
    'house': (11, 1)   # House > Scramble
}

def compute_rdm_spherical(subject_id, functional_results, cope_map, radius=6):
    """Compute RDM stability using spherical ROIs at each session's peak"""
    from scipy.stats import pearsonr
    
    info = ANALYSIS_SUBJECTS[subject_id]
    first_session = info['sessions'][0]
    
    # Get reference image for affine/shape
    roi_dir = BASE_DIR / subject_id / f'ses-{first_session}' / 'ROIs'
    ref_file = roi_dir / f"{info['hemi']}_face_searchmask.nii.gz"
    if not ref_file.exists():
        return None
    
    ref_img = nib.load(ref_file)
    affine = ref_img.affine
    brain_shape = ref_img.shape
    
    results = {}
    
    for roi_key, sessions_data in functional_results.get(subject_id, {}).items():
        sessions = sorted(sessions_data.keys())
        if len(sessions) < 2:
            continue
        
        first_ses = sessions[0]
        last_ses = sessions[-1]
        
        # Get peaks for both sessions
        peak_t1 = sessions_data[first_ses]['centroid']
        peak_t2 = sessions_data[last_ses]['centroid']
        
        # Create spheres
        sphere_t1 = create_sphere(peak_t1, affine, brain_shape, radius)
        sphere_t2 = create_sphere(peak_t2, affine, brain_shape, radius)
        
        rdms = {}
        
        for ses, sphere in [(first_ses, sphere_t1), (last_ses, sphere_t2)]:
            feat_dir = BASE_DIR / subject_id / f'ses-{ses}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
            
            patterns = []
            valid = True
            
            for cat in ['face', 'word', 'object', 'house']:
                cope_num, mult = cope_map[cat]
                z_name = 'zstat1.nii.gz' if ses == first_ses else f'zstat1_ses{first_ses}.nii.gz'
                cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / z_name
                
                if not cope_file.exists():
                    valid = False
                    break
                
                data = nib.load(cope_file).get_fdata() * mult
                pattern = data[sphere]
                
                if len(pattern) == 0 or not np.all(np.isfinite(pattern)):
                    valid = False
                    break
                
                patterns.append(pattern)
            
            if not valid or len(patterns) != 4:
                continue
            
            # Compute RDM
            try:
                corr_matrix = np.corrcoef(patterns)
                rdm = 1 - corr_matrix
                rdms[ses] = rdm
            except:
                continue
        
        # Calculate stability
        if len(rdms) == 2:
            triu_idx = np.triu_indices(4, k=1)
            rdm1 = rdms[first_ses][triu_idx]
            rdm2 = rdms[last_ses][triu_idx]
            
            r, _ = pearsonr(rdm1, rdm2)
            
            category = roi_key.split('_')[1]
            results[category] = r
    
    return results

# Run for all subjects, both COPE maps, multiple radii
print("Computing RDM stability with spherical ROIs...")
print("="*70)

rdm_results = {}

for radius in [6, 8, 10]:
    rdm_results[radius] = {}
    
    for cope_name, cope_map in [('scramble', COPE_SCRAMBLE), ('differential', COPE_DIFFERENTIAL)]:
        rdm_results[radius][cope_name] = {}
        
        for sid in ANALYSIS_SUBJECTS:
            res = compute_rdm_spherical(sid, golarai_functional_final, cope_map, radius)
            if res:
                rdm_results[radius][cope_name][sid] = res

# Summarize
print("\nRDM STABILITY: SPHERICAL ROIs")
print("="*70)

for radius in [6, 8, 10]:
    print(f"\n--- Radius = {radius}mm ---")
    
    for cope_name in ['scramble', 'differential']:
        print(f"\n{cope_name.upper()} contrasts:")
        
        for group in ['OTC', 'nonOTC', 'control']:
            bil_vals = []
            uni_vals = []
            
            for sid, cats in rdm_results[radius][cope_name].items():
                info = ANALYSIS_SUBJECTS.get(sid, {})
                if info.get('group') != group and not (group == 'control' and info.get('patient_status') == 'control'):
                    continue
                
                for cat, val in cats.items():
                    if cat in ['object', 'house']:
                        bil_vals.append(val)
                    else:
                        uni_vals.append(val)
            
            if bil_vals and uni_vals:
                bil_mean = np.mean(bil_vals)
                uni_mean = np.mean(uni_vals)
                print(f"  {group}: Bil={bil_mean:.3f} (n={len(bil_vals)}), Uni={uni_mean:.3f} (n={len(uni_vals)}), Gap={bil_mean-uni_mean:.3f}")

Computing RDM stability with spherical ROIs...

RDM STABILITY: SPHERICAL ROIs

--- Radius = 6mm ---

SCRAMBLE contrasts:
  OTC: Bil=-0.040 (n=10), Uni=0.209 (n=8), Gap=-0.249
  nonOTC: Bil=0.672 (n=18), Uni=0.625 (n=17), Gap=0.047
  control: Bil=0.442 (n=18), Uni=0.603 (n=18), Gap=-0.161

DIFFERENTIAL contrasts:
  OTC: Bil=0.373 (n=10), Uni=0.791 (n=8), Gap=-0.418
  nonOTC: Bil=0.589 (n=18), Uni=0.743 (n=17), Gap=-0.155
  control: Bil=0.711 (n=18), Uni=0.785 (n=18), Gap=-0.073

--- Radius = 8mm ---

SCRAMBLE contrasts:
  OTC: Bil=0.121 (n=10), Uni=0.178 (n=8), Gap=-0.057
  nonOTC: Bil=0.756 (n=18), Uni=0.724 (n=17), Gap=0.032
  control: Bil=0.470 (n=18), Uni=0.701 (n=18), Gap=-0.231

DIFFERENTIAL contrasts:
  OTC: Bil=0.503 (n=10), Uni=0.801 (n=8), Gap=-0.299
  nonOTC: Bil=0.668 (n=18), Uni=0.863 (n=17), Gap=-0.194
  control: Bil=0.735 (n=18), Uni=0.862 (n=18), Gap=-0.128

--- Radius = 10mm ---

SCRAMBLE contrasts:
  OTC: Bil=0.252 (n=10), Uni=0.270 (n=8), Gap=-0.018
  nonOTC: Bil=0.77

In [11]:
# CELL 11: OTC breakdown by category for spherical RDM
print("\nOTC RDM by Category (6mm sphere, scramble contrasts):")
print("-"*50)

for sid, cats in rdm_results[6]['scramble'].items():
    info = ANALYSIS_SUBJECTS.get(sid, {})
    if info.get('group') == 'OTC':
        print(f"{info['code']}: {cats}")


OTC RDM by Category (6mm sphere, scramble contrasts):
--------------------------------------------------
OTC004: {'face': -0.2750938919281908, 'object': -0.177295915684104, 'house': -0.6186535434937752}
OTC008: {'face': 0.5535014593048587, 'object': 0.32733236157370055, 'house': 0.6809066747480204}
OTC010: {'face': 0.6428708216764486, 'word': 0.746786667784844, 'object': 0.6933815285811035, 'house': -0.6294790656823225}
OTC017: {'face': -0.15852141754886462, 'word': -0.6699713730369203, 'object': -0.16530010662513153, 'house': -0.02636252669133305}
OTC021: {'face': 0.6639558718009001, 'word': 0.16796274354889945, 'object': 0.03955119559291048, 'house': -0.5237600789114427}


In [12]:
# CELL 12: Quick stats - Sphere RDM
from scipy.stats import ttest_ind, ttest_rel

print("RDM STABILITY STATS (6mm sphere, differential)")
print("="*50)

# Gather OTC data
otc_bil = []
otc_uni = []

for sid, cats in rdm_results[6]['differential'].items():
    info = ANALYSIS_SUBJECTS.get(sid, {})
    if info.get('group') == 'OTC':
        for cat, val in cats.items():
            if cat in ['object', 'house']:
                otc_bil.append(val)
            else:
                otc_uni.append(val)

t, p = ttest_ind(otc_bil, otc_uni)
print(f"OTC Bil vs Uni: t={t:.3f}, p={p:.4f}")
print(f"  Bil: {np.mean(otc_bil):.3f} ± {np.std(otc_bil)/np.sqrt(len(otc_bil)):.3f}")
print(f"  Uni: {np.mean(otc_uni):.3f} ± {np.std(otc_uni)/np.sqrt(len(otc_uni)):.3f}")

# Same for controls
ctrl_bil = []
ctrl_uni = []

for sid, cats in rdm_results[6]['differential'].items():
    info = ANALYSIS_SUBJECTS.get(sid, {})
    if info.get('patient_status') == 'control':
        for cat, val in cats.items():
            if cat in ['object', 'house']:
                ctrl_bil.append(val)
            else:
                ctrl_uni.append(val)

t, p = ttest_ind(ctrl_bil, ctrl_uni)
print(f"\nControl Bil vs Uni: t={t:.3f}, p={p:.4f}")
print(f"  Bil: {np.mean(ctrl_bil):.3f} ± {np.std(ctrl_bil)/np.sqrt(len(ctrl_bil)):.3f}")
print(f"  Uni: {np.mean(ctrl_uni):.3f} ± {np.std(ctrl_uni)/np.sqrt(len(ctrl_uni)):.3f}")

RDM STABILITY STATS (6mm sphere, differential)
OTC Bil vs Uni: t=-3.134, p=0.0064
  Bil: 0.373 ± 0.104
  Uni: 0.791 ± 0.054

Control Bil vs Uni: t=-0.883, p=0.3835
  Bil: 0.711 ± 0.063
  Uni: 0.785 ± 0.051
